In [1]:
%%configure -f
{
  "conf": {
    "spark.executor.instances": "4",
    "spark.executor.cores": "1",
    "spark.executor.memory": "2g",
    "spark.driver.memory": "2g"
  }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
393,application_1764662801237_0395,pyspark,idle,Link,Link,None,
405,application_1764662801237_0407,pyspark,idle,Link,Link,None,
412,application_1764662801237_0414,pyspark,idle,Link,Link,None,
416,application_1764662801237_0418,pyspark,idle,Link,Link,None,
449,application_1764662801237_0451,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, trim, count
import time

start_time = time.time()

spark = SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.join.preferSortMergeJoin", "true")



crime_2010_2019 = spark.read.option("header", "true").csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
)
crime_2020_2025 = spark.read.option("header", "true").csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"
)
combined_crime = crime_2010_2019.unionByName(crime_2020_2025)

mo_df = spark.read.text(
    "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/MO_codes.txt"
).withColumn("split_cols", split(col("value"), " ", 2)) \
 .withColumn("MO_Code", col("split_cols").getItem(0)) \
 .withColumn("MO_Desc", col("split_cols").getItem(1)) \
 .drop("value", "split_cols")

combined_crime_exploded = combined_crime.withColumn(
    "MO_Code",
    explode(split(col("Mocodes"), " "))
).withColumn("MO_Code", trim(col("MO_Code")))

joined_df_merge = combined_crime_exploded.join(
    mo_df,
    on="MO_Code",
    how="left"
)

mo_summary_merge = (
    joined_df_merge
    .groupBy("MO_Code", "MO_Desc")
    .agg(count("*").alias("Frequency"))
    .filter(
        (col("MO_Code").isNotNull()) & (trim(col("MO_Code")) != "") &
        (col("MO_Desc").isNotNull()) & (trim(col("MO_Desc")) != "")
    )
    .orderBy(col("Frequency").desc())
)

results_merge = mo_summary_merge.collect()



page_size = 50
total_rows = len(results_merge)

print(f"{'MO Code':<10} | {'Description':<50} | {'Frequency':<10}")
print("-" * 80)

for i in range(0, total_rows, page_size):
    chunk = results_merge[i:i + page_size]
    for row in chunk:
        print(f"{row['MO_Code']:<10} | {row['MO_Desc']:<50} | {row['Frequency']:<10}")

end_time = time.time()

print(f"\nSort Merge Join Execution Time: {end_time - start_time:.2f} sec\n")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
450,application_1764662801237_0452,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MO Code    | Description                                        | Frequency 
--------------------------------------------------------------------------------
0344       | Removes vict property                              | 1002900   
1822       | Stranger                                           | 548422    
0416       | Hit-Hit w/ weapon                                  | 404773    
0329       | Vandalized                                         | 377536    
0913       | Victim knew Suspect                                | 278618    
2000       | Domestic violence                                  | 256188    
1300       | Vehicle involved                                   | 219082    
0400       | Force used                                         | 213165    
1402       | Evidence Booked (any crime)                        | 177470    
1609       | Smashed                                            | 131229    
1309       | Susp uses vehicle                                  | 122108